<font size="5">**Project Summary**</font><br>
In this competition, the main task is to do surface time series classification. 1d convolution is a good technique when doing the time series classification job. So I apply 1d convolutional neural network to solve this problem. The whole code is written in Pytorch.

<font size="5">**External libs we are using**</font><br>

In [939]:
# libs we are using
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

<font size="5">**Data preparations**</font><br>

In this project, I use the raw data as input of the network. I concatenated all datasets into one single numpy array. The first 3810 rows are training data with given labels, the rest 3816 rows are testing data where labels are not given. In total raw data has 7816 rows. In addition to that, we split the raw training data into training data and validation data by the ratio of 80/20.


I create three datasets and data loaders for them to make the data ready for model's training. The process is straightforward. I split the labelled data into two subsets, and keep testing data as is. Also, I convert Numpy arrays into `torch.tensor` objects of proper type (float for samples, and long - for targets).


In [940]:
def create_datasets(data, target, train_size, valid_pct=0.2, seed=None):
    """Converts NumPy arrays into PyTorch datsets.
    
    Three datasets are created in total:
        * training dataset
        * validation dataset
        * testing (un-labelled) dataset

    """
    raw =  data
    sz = train_size
    idx = np.arange(sz)
    trn_idx, val_idx = train_test_split(
        idx, test_size=valid_pct, random_state=seed)
     
    trn_ds = TensorDataset(
        torch.tensor(raw[:sz][trn_idx]).float(), 
        torch.tensor(target[:sz][trn_idx]).long())
    val_ds = TensorDataset(
        torch.tensor(raw[:sz][val_idx]).float(), 
        torch.tensor(target[:sz][val_idx]).long())
    tst_ds = TensorDataset(
        torch.tensor(raw[sz:]).float(), 
        torch.tensor(target[sz:]).long())
    return trn_ds, val_ds, tst_ds

# We use this function to divide the data
def create_datasets2(data_arr, train_size, valid_pct=0.2, seed=None):
    """Converts NumPy arrays into PyTorch datsets.
    
    Three datasets are created in total:
        * training dataset
        * validation dataset
        * testing (un-labelled) dataset

    """
    raw =  data_arr
    sz = train_size

    idx = np.arange(sz)
    print("total data length")
    print(len(idx))
    trn_idx, tst_idx = train_test_split(idx, test_size=0.2, random_state=1)
    trn_idx, val_idx = train_test_split(trn_idx, test_size=0.25, random_state=1)
    
    print("input one dimension shape")
    print(raw[0].shape)
    print("training data length")
    print(len(trn_idx))
    print("validation data le")
    print(len(val_idx))
    print("testing d")
    print(len(tst_idx))
    
    trn_ds = TensorDataset(
        torch.tensor(raw[0][:sz][trn_idx]).float(),
        torch.tensor(raw[1][:sz][trn_idx]).float(),
        torch.tensor(raw[2][:sz][trn_idx]).float(),
        torch.tensor(raw[3][:sz][trn_idx]).float(),
        torch.tensor(raw[4][:sz][trn_idx]).float(),
        torch.tensor(raw[5][:sz][trn_idx]).float(),
        torch.tensor(raw[6][:sz][trn_idx]).float(),
        torch.tensor(raw[7][:sz][trn_idx]).float(),
        torch.tensor(raw[8][:sz][trn_idx]).float(),
        torch.tensor(target[:sz][trn_idx]).long())
    
    val_ds = TensorDataset(
        torch.tensor(raw[0][:sz][val_idx]).float(),
        torch.tensor(raw[1][:sz][val_idx]).float(), 
        torch.tensor(raw[2][:sz][val_idx]).float(), 
        torch.tensor(raw[3][:sz][val_idx]).float(), 
        torch.tensor(raw[4][:sz][val_idx]).float(), 
        torch.tensor(raw[5][:sz][val_idx]).float(), 
        torch.tensor(raw[6][:sz][val_idx]).float(), 
        torch.tensor(raw[7][:sz][val_idx]).float(), 
        torch.tensor(raw[8][:sz][val_idx]).float(), 
        torch.tensor(target[:sz][val_idx]).long())
    
    tst_ds = TensorDataset(
        torch.tensor(raw[0][:sz][tst_idx]).float(),
        torch.tensor(raw[1][:sz][tst_idx]).float(),
        torch.tensor(raw[2][:sz][tst_idx]).float(),
        torch.tensor(raw[3][:sz][tst_idx]).float(),
        torch.tensor(raw[4][:sz][tst_idx]).float(),
        torch.tensor(raw[5][:sz][tst_idx]).float(),
        torch.tensor(raw[6][:sz][tst_idx]).float(),
        torch.tensor(raw[7][:sz][tst_idx]).float(),
        torch.tensor(raw[8][:sz][tst_idx]).float(),
        torch.tensor(target[:sz][tst_idx]).long())
    
    return trn_ds, val_ds, tst_ds

def create_loaders(data, bs=128, jobs=0):
    """Wraps the datasets returned by create_datasets function with data loaders."""
    
    trn_ds, val_ds, tst_ds = data
    trn_dl = DataLoader(trn_ds, batch_size=bs, shuffle=True, num_workers=jobs)
    val_dl = DataLoader(val_ds, batch_size=bs, shuffle=False, num_workers=jobs)
    tst_dl = DataLoader(tst_ds, batch_size=bs, shuffle=False, num_workers=jobs)
    return trn_dl, val_dl, tst_dl

<font size="5">**Neural Network Architecture**</font><br>

In [941]:
class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)

In [942]:
class Surface_Classifier(nn.Module):
    def __init__(self, raw_ni, no, drop=.5):
        super().__init__()
        
        self.conv1d_channel_0 = nn.Sequential(
            nn.Conv1d(1, 6, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            nn.MaxPool1d(kernel_size=3),
            
            nn.BatchNorm1d(6),
            
            nn.Conv1d(6,10, 8, 2, 1),
            nn.ReLU(),
            nn.Dropout(),
            nn.MaxPool1d(kernel_size=2),
            
            nn.BatchNorm1d(10),
            
            nn.Conv1d(10, 16, 4, 2, 1),
            nn.ReLU(), 
            nn.Dropout(),
            
            
            nn.Conv1d(16, 32, 3, 2, 1),
            nn.ReLU(), 
            nn.Dropout(),
            
            nn.Conv1d(32, 64, 3, 2, 1),
            nn.ReLU(), 
            nn.Dropout(),
        )
            
            
            
           
        
        self.conv1d_channel_1 = nn.Sequential(
            nn.Conv1d(1, 6, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=3),
            
            nn.BatchNorm1d(6),
            
            nn.Conv1d(6,10, 8, 2, 1),
            nn.ReLU(),
            nn.Dropout(),
            nn.MaxPool1d(kernel_size=2),
            
            nn.BatchNorm1d(10),
            
            nn.Conv1d(10, 16, 4, 2, 1),
            nn.ReLU(),
            nn.Dropout(),
            
            
            nn.Conv1d(16, 32, 3, 2, 1),
            nn.ReLU(), 
            nn.Dropout(),
            
            nn.Conv1d(32, 64, 3, 2, 1),
            nn.ReLU(), 
            nn.Dropout(),
            
            
            
        )
        
        self.conv1d_channel_2 = nn.Sequential(
            nn.Conv1d(1, 6, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=3),
            
            nn.BatchNorm1d(6),
            
            nn.Conv1d(6,10, 8, 2, 1),
            nn.ReLU(),
            nn.Dropout(),
            nn.MaxPool1d(kernel_size=2),
            
            nn.BatchNorm1d(10),
            
            nn.Conv1d(10, 16, 4, 2, 1),
            nn.ReLU(),
            nn.Dropout(),
        
            nn.Conv1d(16, 32, 3, 2, 1),
            nn.ReLU(), 
            nn.Dropout(),
            
            nn.Conv1d(32, 64, 3, 2, 1),
            nn.ReLU(), 
            nn.Dropout(),
            
         
           
        )
        
        self.conv1d_channel_3 = nn.Sequential(
            nn.Conv1d(1, 6, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            nn.MaxPool1d(kernel_size=3),
            nn.BatchNorm1d(6),
            
            nn.Conv1d(6,10, 8, 2, 1),
            nn.ReLU(),
            nn.Dropout(),
            nn.MaxPool1d(kernel_size=2),
            nn.BatchNorm1d(10),
            
            nn.Conv1d(10, 16, 4, 2, 1),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.Conv1d(16, 32, 3, 2, 1),
            nn.ReLU(), 
            nn.Dropout(),
            
            nn.Conv1d(32, 64, 3, 2, 1),
            nn.ReLU(), 
            nn.Dropout(),
           
               
        )
        
        self.conv1d_channel_4 = nn.Sequential(
            nn.Conv1d(1, 6, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            nn.MaxPool1d(kernel_size=3),
            nn.BatchNorm1d(6),
            
            nn.Conv1d(6,10, 8, 2, 1),
            nn.ReLU(),
            nn.Dropout(),
            nn.MaxPool1d(kernel_size=2),
            nn.BatchNorm1d(10),
            
            nn.Conv1d(10, 16, 4, 2, 1),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.Conv1d(16, 32, 3, 2, 1),
            nn.ReLU(), 
            nn.Dropout(),
            
            nn.Conv1d(32, 64, 3, 2, 1),
            nn.ReLU(), 
            nn.Dropout(),
            
            
            
           
            
           
        )
        
        self.conv1d_channel_5 = nn.Sequential(
            nn.Conv1d(1, 6, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            nn.MaxPool1d(kernel_size=3),
            nn.BatchNorm1d(6),
        
            nn.Conv1d(6,10, 8, 2, 1),
            nn.ReLU(),
            nn.Dropout(),
            nn.MaxPool1d(kernel_size=2),
            nn.BatchNorm1d(10),
            
            nn.Conv1d(10, 16, 4, 2, 1),
            nn.ReLU(),
            nn.Dropout(),
           
            nn.Conv1d(16, 32, 3, 2, 1),
            nn.ReLU(), 
            nn.Dropout(),
            
            nn.Conv1d(32, 64, 3, 2, 1),
            nn.ReLU(), 
            nn.Dropout(),
         
            
        )
        
        self.conv1d_channel_6 = nn.Sequential(
            nn.Conv1d(1, 6, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            nn.MaxPool1d(kernel_size=3),
            nn.BatchNorm1d(6),
            
            nn.Conv1d(6,10, 8, 2, 1),
            nn.ReLU(),
            nn.Dropout(),
            nn.MaxPool1d(kernel_size=2),
            nn.BatchNorm1d(10),
            
            nn.Conv1d(10, 16, 4, 2, 1),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.Conv1d(16, 32, 3, 2, 1),
            nn.ReLU(), 
            nn.Dropout(),
            
            nn.Conv1d(32, 64, 3, 2, 1),
            nn.ReLU(), 
            nn.Dropout(),
            
            
           
           
        )
        
        self.conv1d_channel_7 = nn.Sequential(
            nn.Conv1d(1, 6, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            nn.MaxPool1d(kernel_size=3),
            nn.BatchNorm1d(6),
            nn.Conv1d(6,10, 8, 2, 1),
            nn.ReLU(),
            nn.Dropout(),
            nn.MaxPool1d(kernel_size=2),
            nn.BatchNorm1d(10),
            
            nn.Conv1d(10, 16, 4, 2, 1),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.Conv1d(16, 32, 3, 2, 1),
            nn.ReLU(), 
            nn.Dropout(),
            
            nn.Conv1d(32, 64, 3, 2, 1),
            nn.ReLU(), 
            nn.Dropout(),
            
        )
            

       
        
        self.conv1d_channel_8 = nn.Sequential(
            nn.Conv1d(1, 6, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            nn.MaxPool1d(kernel_size=3),
            nn.BatchNorm1d(6),
            
            nn.Conv1d(6,10, 8, 2, 1),
            nn.ReLU(),
            nn.Dropout(),
            nn.BatchNorm1d(10),
            
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(10, 16, 4, 2, 1),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.Conv1d(16, 32, 3, 2, 1),
            nn.ReLU(), 
            nn.Dropout(),
            
            nn.Conv1d(32, 64, 3, 2, 1),
            nn.ReLU(), 
            nn.Dropout(),
            
        )

            

        self.dense = nn.Sequential(
            nn.Linear(576, 288),  nn.ReLU(),nn.Dropout(),
            nn.Linear(288, 64),  nn.ReLU(),nn.Dropout(),
            nn.Linear(64, no),  nn.ReLU(),
        )
        
            

    def forward(self, t_channel_0, t_channel_1, t_channel_2, t_channel_3, t_channel_4, t_channel_5, t_channel_6, t_channel_7, t_channel_8):
        conv1d_out_channel_0 = self.conv1d_channel_0(t_channel_0)
        conv1d_out_channel_1 = self.conv1d_channel_1(t_channel_1)
        conv1d_out_channel_2 = self.conv1d_channel_2(t_channel_2)
        conv1d_out_channel_3 = self.conv1d_channel_3(t_channel_3)
        conv1d_out_channel_4 = self.conv1d_channel_4(t_channel_4)
        conv1d_out_channel_5 = self.conv1d_channel_5(t_channel_5)
        conv1d_out_channel_6 = self.conv1d_channel_6(t_channel_6)
        conv1d_out_channel_7 = self.conv1d_channel_7(t_channel_7)
        conv1d_out_channel_8 = self.conv1d_channel_8(t_channel_8)
        
        t_in = torch.cat([conv1d_out_channel_0,conv1d_out_channel_1, conv1d_out_channel_2, conv1d_out_channel_3, conv1d_out_channel_4, conv1d_out_channel_5, conv1d_out_channel_6, conv1d_out_channel_7, conv1d_out_channel_8], dim=1)
        res = t_in.view(t_in.size(0), -1)
        out = self.dense(res)
        return out
        

<font size="5">**Initial setup for random seed and project path**</font><br>

In [943]:

# set up the seed
seed = 1
np.random.seed(seed)

# path to sample submission
sample = Path.cwd().parent/'input'/'career-con-2019'/'sample_submission.csv'

ROOT = Path.cwd().parent/'input'/'career-con-2019-preprocessed-data'
raw_arr = np.load(ROOT/'feat.npy').transpose(0, 2, 1)
target = np.load(ROOT/'target.npy')

raw_dim_data = [None]*9
#print(raw_dim_data)
for i in range(0, 9):
    raw_dim_data[i] = raw_arr[:,i,:]
#    print("raw data shape")
    
    raw_dim_data[i] = raw_dim_data[i].reshape([7626,1,128])
#    print(raw_dim_data[i].shape)
    
# print("raw array shape")
# print(raw_arr.shape)
# print("label array shape")
# print(target.shape)

trn_sz = 3810  # only the first `trn_sz` rows in each array include labelled data
#datasets = create_datasets((raw_arr), target, trn_sz, seed=seed)
datasets = create_datasets2((raw_dim_data), trn_sz, seed=seed)


# make sure that we run on a proper device (not relevant for Kaggle kernels but helpful in Jupyter sessions)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


trn_dl, val_dl, tst_dl = create_loaders(datasets, bs=256)

total data length
3810
input one dimension shape
(7626, 1, 128)
training data length
2286
validation data le
762
testing d
762


<font size="5">**Training model**</font><br>
Now everything is ready to create a training loop and see if our model works. For each training epoch, the loop performs the following actions:
1. train model on the `trn_ds` dataset;
2. verify quality on the `val_ds` dataset;
3. check if the quality improved since previous epoch, and if so, save the model's weights onto disk;
4. in case if the model's quality isn't impoving for `patience` epochs, the training is stopped.
Also, the code tracks loss and accuracy history, and prints current scores with exponentially increasing logging frequency, i.e., only at 1, 2, 4, 8... epochs.

In [ ]:
raw_feat = raw_arr.shape[1]


lr = 0.001
n_epochs = 10000
iterations_per_epoch = len(trn_dl)
num_classes = 9
best_acc = 0
patience, trials = 500, 0
base = 1
step = 2
loss_history = []
acc_history = []

model = Surface_Classifier(raw_feat, num_classes).to(device)
model.cuda()
criterion = nn.CrossEntropyLoss(reduction='sum')
opt = optim.Adam(model.parameters(), lr=lr)

print('Start model training')

for epoch in range(1, n_epochs + 1):
    
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(trn_dl):
        #x_raw, y_batch = [t.to(device) for t in batch]
        x_channel_0, x_channel_1,  x_channel_2,  x_channel_3,  x_channel_4,  x_channel_5,  x_channel_6,  x_channel_7,  x_channel_8, y_batch = [t.to(device) for t in batch]
        opt.zero_grad()
        
#         print("channel 0 type")
#         print(type(x_channel_0))
#         print("channel 0 shape")
#         print(x_channel_0.shape)
#         print("batch type")
#         print(type(batch))
#         print(len(batch))
#         print(batch[0].shape)
#         print(batch[9].shape)

        out = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
        
        
#         out = []
#         with torch.no_grad():
#             for x_channel_0, x_channel_1,  x_channel_2,  x_channel_3,  x_channel_4,  x_channel_5,  x_channel_6,  x_channel_7,  x_channel_8, y_batch in batch:
#                 output = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
#                 out.append(output.detach())
#         out = torch.cat(out)
        

    
        loss = criterion(out, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        opt.step()
        
    epoch_loss /= trn_sz
    loss_history.append(epoch_loss)
    
    model.eval()
    correct, total = 0, 0
    
    for batch in val_dl:
        x_channel_0, x_channel_1,  x_channel_2,  x_channel_3,  x_channel_4,  x_channel_5,  x_channel_6,  x_channel_7,  x_channel_8, y_batch = [t.to(device) for t in batch]
        
        out = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()
    
    acc = correct / total
    acc_history.append(acc)

    if epoch % base == 0:
        print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')
        base *= step
    print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')

    
    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break
            
print('Done!')

Start model training
Epoch:   1. Loss: 1.3192. Acc.: 15.09%
Epoch:   1. Loss: 1.3192. Acc.: 15.09%
Epoch 1 best model saved with accuracy: 15.09%
Epoch:   2. Loss: 1.3147. Acc.: 15.09%
Epoch:   2. Loss: 1.3147. Acc.: 15.09%
Epoch:   3. Loss: 1.3015. Acc.: 15.09%
Epoch:   4. Loss: 1.2751. Acc.: 21.78%
Epoch:   4. Loss: 1.2751. Acc.: 21.78%
Epoch 4 best model saved with accuracy: 21.78%
Epoch:   5. Loss: 1.2563. Acc.: 22.05%
Epoch 5 best model saved with accuracy: 22.05%
Epoch:   6. Loss: 1.2592. Acc.: 21.78%
Epoch:   7. Loss: 1.2455. Acc.: 21.78%
Epoch:   8. Loss: 1.2435. Acc.: 23.62%
Epoch:   8. Loss: 1.2435. Acc.: 23.62%
Epoch 8 best model saved with accuracy: 23.62%
Epoch:   9. Loss: 1.2330. Acc.: 27.69%
Epoch 9 best model saved with accuracy: 27.69%
Epoch:  10. Loss: 1.2118. Acc.: 27.82%
Epoch 10 best model saved with accuracy: 27.82%
Epoch:  11. Loss: 1.1742. Acc.: 33.60%
Epoch 11 best model saved with accuracy: 33.60%
Epoch:  12. Loss: 1.1397. Acc.: 32.28%
Epoch:  13. Loss: 1.1176

Epoch: 160. Loss: 0.7683. Acc.: 51.05%
Epoch: 161. Loss: 0.7880. Acc.: 50.66%
Epoch: 162. Loss: 0.7667. Acc.: 50.79%
Epoch: 163. Loss: 0.7829. Acc.: 51.18%
Epoch: 164. Loss: 0.7777. Acc.: 49.87%
Epoch: 165. Loss: 0.7964. Acc.: 51.57%
Epoch: 166. Loss: 0.7709. Acc.: 49.08%
Epoch: 167. Loss: 0.7791. Acc.: 51.97%
Epoch: 168. Loss: 0.7696. Acc.: 48.69%
Epoch: 169. Loss: 0.7696. Acc.: 50.26%
Epoch: 170. Loss: 0.7791. Acc.: 51.44%
Epoch: 171. Loss: 0.7812. Acc.: 52.10%
Epoch: 172. Loss: 0.7849. Acc.: 47.51%
Epoch: 173. Loss: 0.7604. Acc.: 50.66%
Epoch: 174. Loss: 0.7659. Acc.: 51.84%
Epoch: 175. Loss: 0.7695. Acc.: 51.84%
Epoch: 176. Loss: 0.7597. Acc.: 50.79%
Epoch: 177. Loss: 0.7445. Acc.: 51.84%
Epoch: 178. Loss: 0.7530. Acc.: 50.79%
Epoch: 179. Loss: 0.7661. Acc.: 51.97%
Epoch: 180. Loss: 0.7469. Acc.: 51.44%
Epoch: 181. Loss: 0.7735. Acc.: 50.13%
Epoch: 182. Loss: 0.7611. Acc.: 51.05%
Epoch: 183. Loss: 0.7623. Acc.: 47.77%
Epoch: 184. Loss: 0.7469. Acc.: 53.02%
Epoch: 185. Loss: 0.7627.

Epoch: 368. Loss: 0.6666. Acc.: 49.34%
Epoch: 369. Loss: 0.6840. Acc.: 49.48%
Epoch: 370. Loss: 0.6780. Acc.: 48.29%
Epoch: 371. Loss: 0.6962. Acc.: 46.98%
Epoch: 372. Loss: 0.6833. Acc.: 48.43%
Epoch: 373. Loss: 0.6968. Acc.: 50.66%
Epoch: 374. Loss: 0.7084. Acc.: 49.48%
Epoch: 375. Loss: 0.6778. Acc.: 50.00%
Epoch: 376. Loss: 0.6893. Acc.: 50.00%
Epoch: 377. Loss: 0.6827. Acc.: 50.92%
Epoch: 378. Loss: 0.6782. Acc.: 49.74%
Epoch: 379. Loss: 0.6784. Acc.: 49.21%
Epoch: 380. Loss: 0.6926. Acc.: 50.00%
Epoch: 381. Loss: 0.6795. Acc.: 50.13%
Epoch: 382. Loss: 0.6949. Acc.: 51.05%
Epoch: 383. Loss: 0.6778. Acc.: 49.08%
Epoch: 384. Loss: 0.6704. Acc.: 49.74%
Epoch: 385. Loss: 0.6899. Acc.: 49.48%
Epoch: 386. Loss: 0.6799. Acc.: 51.05%
Epoch: 387. Loss: 0.6717. Acc.: 49.21%
Epoch: 388. Loss: 0.6852. Acc.: 48.43%
Epoch: 389. Loss: 0.6590. Acc.: 48.69%
Epoch: 390. Loss: 0.6745. Acc.: 46.85%
Epoch: 391. Loss: 0.6687. Acc.: 49.21%
Epoch: 392. Loss: 0.6756. Acc.: 51.57%
Epoch: 393. Loss: 0.6882.

<font size="5">**Compute result on testing data**</font><br>

In [ ]:
test_results = []
model.load_state_dict(torch.load('best.pth'))
model.eval()

# for batch in val_dl:
#     x_raw, y_batch = [t.to(device) for t in batch]
#     out = model(x_raw)
#     preds = F.log_softmax(out, dim=1).argmax(dim=1)
#     total += y_batch.size(0)
#     correct += (preds == y_batch).sum().item()
#     acc = correct / total
#     acc_history.append(acc)

# for batch in tst_dl:
#     x, y_batche = [t.to(device) for t in (batch)]
#     out = model(x)
#     y_hat = F.log_softmax(out, dim=1).argmax(dim=1)
#     test_results.extend(y_hat.tolist())
    
for batch in tst_dl:
        x_raw, y_batch = [t.to(device) for t in batch]
        out = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()
    
acc = correct / total
print("accurancy on test data: "+str(acc))

<font size="5">**Store final result**</font><br>

In [ ]:
'''
submit = pd.read_csv(sample)
enc = joblib.load(ROOT/'encoder.model')
submit['surface'] = enc.inverse_transform(test_results)
submit.to_csv('submit_base.csv', index=None)
print("store result successfully!")
'''